In [ ]:
# import tensorflow as tf
import prepare_fnn_data.py as mp

In [ ]:
dat = mp.data_wrapper()

num_epochs = 5000
batchSize = 64

batch_gen = dat.batch_gen#batch_generator(batch_size=batchSize)

dataset = tf.data.Dataset.\
    from_generator(batch_gen.generate, (tf.float32,tf.float32),
                   output_shapes= (tf.TensorShape([1050]), 
                                   tf.TensorShape([3])))                                                     
dataset = dataset.shuffle(buffer_size = batchSize*10) 
dataset = dataset.repeat(num_epochs).batch(batchSize)
dataset = dataset.prefetch(buffer_size = 2)
data_source =  dataset.make_one_shot_iterator()
X_batch, Y_batch = data_source.get_next()

In [ ]:
with tf.name_scope("dnn"):
    global X_batch, loss, prediction
    hidden1 = tf.keras.layers.Dense(1024,activation='elu')(X_batch)
    hidden2 = tf.keras.layers.Dense(2048,activation='relu')(hidden1)
    dropout = tf.keras.layers.Dropout(rate=0.2)(hidden2)
    hidden3 =  tf.keras.layers.Dense(1024,activation='elu')(dropout)
    prediction = tf.keras.layers.Dense(3)(hidden2)
    
with tf.name_scope("loss"):
    loss = tf.losses.mean_squared_error(Y_batch, prediction)

In [ ]:
print(prediction.name)
print(X_batch.name)

In [ ]:
learning_rate = 0.001
with tf.name_scope("train"):
    global training_op
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss) 

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    init.run()
    for epoch in range(num_epochs):
        for iteration in range(batch_gen.samples// batchSize):
            _,loss_value = sess.run([training_op,loss])
            if iteration % 500 == 0:
                print("Epoch " + str(epoch) + " Step " + str(iteration) + " loss " + str(loss_value))
        if epoch % 20 == 0:
            save_path = saver.save(sess, "../models/my_model_" + str(epoch) + ".ckpt")